# Get Huggingface pretrained model

In [10]:
!pip install transformers --upgrade --quiet

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


In [11]:
import transformers
import torch
from transformers import pipeline

## Get pretrained model, test and save it

In [12]:
pretrained_classifier = pipeline("sentiment-analysis")

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [13]:
pretrained_classifier([
    "I really like this place!",
    "The food was bad!"
])

[{'label': 'POSITIVE', 'score': 0.9998599290847778},
 {'label': 'NEGATIVE', 'score': 0.9997907280921936}]

In [14]:
pretrained_classifier.save_pretrained("./model/")

## Check if we can successfully load our saved model

In [15]:
local_classifier = pipeline('sentiment-analysis', model="./model/", tokenizer="./model/")

In [16]:
local_classifier([
    "I really like this place!",
    "The food was bad!"
])

[{'label': 'POSITIVE', 'score': 0.9998599290847778},
 {'label': 'NEGATIVE', 'score': 0.9997907280921936}]

ok this works!

## Build container for Lambda

Let's build a container that holds our huggingface model 
and our lambda code that handles requests and forwards the data to our huggingface model.

In [18]:
print("look at the versions for our dependencies:")
print("")
!python --version
print(f"Torch version {torch.__version__}")
print(f"Transformers version {transformers.__version__}")

look at the versions for our dependencies:

Python 3.6.13
Torch version 1.7.1
Transformers version 4.9.2


### Create requirements.txt with dependencies

In [20]:
%%writefile requirements.txt
--find-links  https://download.pytorch.org/whl/torch_stable.html 
 
torch==1.7.1+cpu
transformers==4.9.2

Overwriting requirements.txt


### Create dockerfile
https://docs.aws.amazon.com/lambda/latest/dg/images-create.html

- start from the official lambda docker container.
- copy the requirements.txt to the docker container and isntall.
- copy our model we saved locally in the model/ folder to the docker container
- execcute the lambda handler function on start

In [21]:
%%writefile Dockerfile
FROM public.ecr.aws/lambda/python:3.8

COPY requirements.txt ./requirements.txt
RUN pip install -r requirements.txt 

COPY ./model/   ./model/
COPY ./app/app.py   ./
CMD ["app.handler"]

Overwriting Dockerfile


### Build dockerfile and push to ECR

In [22]:
!pip install sagemaker-studio-image-build --quiet

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


make sure this role has a trust relationship with codebuild

https://github.com/aws-samples/sagemaker-studio-image-build-cli

In [ ]:
!sm-docker build . --repository huggingface-on-lambda:1.0

...[Container] 2021/08/21 07:29:27 Waiting for agent ping

[Container] 2021/08/21 07:29:30 Waiting for DOWNLOAD_SOURCE
[Container] 2021/08/21 07:29:33 Phase is DOWNLOAD_SOURCE
[Container] 2021/08/21 07:29:33 CODEBUILD_SRC_DIR=/codebuild/output/src864854616/src
[Container] 2021/08/21 07:29:33 YAML location is /codebuild/output/src864854616/src/buildspec.yml
[Container] 2021/08/21 07:29:33 Processing environment variables
[Container] 2021/08/21 07:29:34 No runtime version selected in buildspec.
[Container] 2021/08/21 07:29:34 Moving to directory /codebuild/output/src864854616/src
[Container] 2021/08/21 07:29:34 Registering with agent
[Container] 2021/08/21 07:29:34 Phases found in YAML: 3
[Container] 2021/08/21 07:29:34  PRE_BUILD: 9 commands
[Container] 2021/08/21 07:29:34  BUILD: 4 commands
[Container] 2021/08/21 07:29:34  POST_BUILD: 3 commands
[Container] 2021/08/21 07:29:34 Phase complete: DOWNLOAD_SOURCE State: SUCCEEDED
[Container] 2021/08/21 07:29:34 Phase context status code:  M

You should consider upgrading via the '/var/lang/bin/python3.8 -m pip install --upgrade pip' command.
Removing intermediate container bf95240df9f7
 ---> 2180e6aaa55c
Step 4/6 : COPY ./model/   ./model/
 ---> abddf366c4f6
Step 5/6 : COPY ./app/app.py   ./
 ---> 1d136bf1feac
Step 6/6 : CMD ["app.handler"]
 ---> Running in 654ce2c510ff
Removing intermediate container 654ce2c510ff
 ---> f8a7eb8182fa
Successfully built f8a7eb8182fa
Successfully tagged huggingface-on-lambda:1.0

[Container] 2021/08/21 07:30:48 Running command docker tag $IMAGE_REPO_NAME:$IMAGE_TAG $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com/$IMAGE_REPO_NAME:$IMAGE_TAG

[Container] 2021/08/21 07:30:48 Phase complete: BUILD State: SUCCEEDED
[Container] 2021/08/21 07:30:48 Phase context status code:  Message:
[Container] 2021/08/21 07:30:48 Entering phase POST_BUILD
[Container] 2021/08/21 07:30:48 Running command echo Build completed on `date`
Build completed on Sat Aug 21 07:30:48 UTC 2021

[Container] 2021/08/21

## Deploy Container within a Lambda Function

Create a serverless.yml file that points to the docker image uri + digest.

We expect a POST request on the path "prediction/"

More configuration can be found in the serverless file ...

In [24]:
ACCOUNT_NUMBER = ""
REGION = ""
# {ACCOUNT_NUMBER}.dkr.ecr.{REGION}.amazonaws.com/huggingface-on-lambda
image_uri= ""
# sha256:f0e2ae3aee2cceb6d93d...
image_digest = ""

__serverless file__:

In [ ]:
print(f"""
service: huggingface-on-lambda

provider:
  name: aws
  region: eu-west-1 

functions:
  huggingface:
    image: {image_uri}@{image_digest}
    # 2 minutes before we throw a timeout
    timeout: 120
    # have 1 hot lambda available
    provisionedConcurrency: 1
    # our model is less than 1GB, so 1024MB is enough
    memorySize: 1024 
    events:
      - http:
          path: prediction
          method: post

""")

open an AWS cloudshell and install the serverless framework

    npm install serverless

create the serverless.yml file in the cloudshell
    
    cat > serverless.yml

paste the contents and press ctrl + c to close and save the file.
To deploy the stack run

    npm install --prefix ./ serverless
    node_modules/serverless/bin/serverless.js deploy
    
    
            Output:
    
            Serverless: Packaging service...
            Serverless: WARNING: Function huggingface has timeout of 120 seconds, however, 
            it's attached to API Gateway so it's automatically limited to 30 seconds.
            Serverless: Uploading CloudFormation file to S3...
            Serverless: Uploading artifacts...
            Serverless: Validating template...
            Serverless: Updating Stack...
            Serverless: Checking Stack update progress...
            ........................
            Serverless: Stack update finished...
            Service Information
            service: huggingface-on-lambda
            stage: dev
            region: eu-west-1
            stack: huggingface-on-lambda-dev
            resources: 12
            api keys:
              None
            endpoints:
              POST - https://8af9ar02gi.execute-api.eu-west-1.amazonaws.com/dev/prediction
            functions:
              huggingface: huggingface-on-lambda-dev-huggingface
            layers:
              None
            Serverless: Removing old service artifacts from S3...

if deploy succeeds, call the endpoint

    curl -d '{"data":"some very much wow positive text!"}' -H "Content-Type: application/json" -X POST https://8af9ar02gi.execute-api.eu-west-1.amazonaws.com/dev/prediction
            
            Output:
            
            [{"label": "POSITIVE", "score": 0.9998674392700195}]